- Table of contents
- [Todo App](#todo-app)
  - [Create prompt](#create-prompt)

# Todo App

- It can do following things on the behalf of user:
  - Create Todo
  - Update Todo
  - Delete Todo
  - Get Todo

## Create Prompt

- Here I am going to give context how they behave on user input.
- I use [**`few_shot_prompting`**](../../05_few_shot_prompting/few_shot_prompting.md), that them to learn from examples.
- I will give them few examples of how to behave on user input.

In [63]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage

system_message = """
You are a helpful assistant for todo management (Create, Update, Delete, Get).
1.  **Analyze Intent:** Determine the user's primary action: Create, Update, Delete, or Get.
2.  **Specify Todos:** For Update/Delete operations, ensure the correct todo is targeted; ask to clarify if needed (e.g., if multiple match or none are found). For Get, determine if all todos or a filtered list is needed.
3.  **Batch Operations:** Recognize and process requests involving multiple todos.
4.  **Action & Feedback:** Execute the request and confirm completion or explain issues.
5. **Choose correct tool:** Determine the correct tool to call such as Create, Update, Delete or Get.
"""
examples = [
    HumanMessage(
        content="Create a new todo item for grocery shopping. I need to buy milk and eggs. it is due tomorrow. and have a priority of 1.",
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "create_todo",
                "args": {
                    "name": "Grocery shopping",
                    "description": "Buy milk and eggs",
                    "due_date": "tomorrow",
                    "priority": 1
                },
                "id": "1"
            }
        ]
    ),
    ToolMessage("", tool_call_id="1"),
    HumanMessage(
        content="Update the grocery shopping todo item to include bread and change the due date to next week.",
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "update_todo",
                "args": {
                    "id": "1",
                    "description": "Buy milk, eggs, and bread",
                    "due_date": "next week"
                },
                "id": "2"
            }
        ]
    ),
    ToolMessage("", tool_call_id="2"),
    HumanMessage(
        content="Can you add a new todo item for laundry? It should be due next Monday. and also add going to the gym.",
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "create_todo",
                "args": {
                    "name": "Laundry",
                    "description": "",
                    "due_date": "next Monday",
                    "priority": 1
                },
                "id": "3"
            },
            {
                "name": "create_todo",
                "args": {
                    "name": "Going to the gym",
                    "description": "",
                    "due_date": "",
                    "priority": 1
                },
                "id": "4"
            }
        ]
    ),
    ToolMessage("", tool_call_id="3"),
    ToolMessage("", tool_call_id="4"),
]

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=system_message),
        *examples,
        ("human", "{input}")
    ]
)

## Creating Output Structure

### Create tool arguments Structure

In [64]:
from typing import List, Optional, Union
from pydantic import BaseModel, Field

# Todo Schema


class TodoSchema(BaseModel):
    id: str = Field(description="Ids of the todo item.")
    name: str = Field(description="Name of the todo.")
    description: Optional[str] = Field(
        description="Description of the todo.", default=None)
    due_date: Optional[str] = Field(
        description="Due data of todo.", default=None)
    priority: Optional[int] = Field(
        description="Priority of todo.", default=None)


# Create todo schema
class CreateTodoSchema(BaseModel):
    name: str = Field(description="Name of the todo")
    description: Optional[str] = Field(description="Description of the todo")
    due_date: Optional[str] = Field(description="Due data of todo")
    priority: Optional[int] = Field(description="Priority of todo")


# Update todo schema
class UpdateTodoSchema(BaseModel):
    id: str = Field(description="Id of the todo that we want to update.")
    name: Optional[str] = Field(description="Update name of the todo")
    description: Optional[str] = Field(
        description="Updated description of the todo")
    due_date: Optional[str] = Field(description="updated due date of the todo")
    priority: Optional[int] = Field(description="Updated priority of the todo")


# Delete todo schema
class DeleteTodoSchema(BaseModel):
    id: Union[str, List[str]] = Field(
        description="Id of the todo that we want to delete.")

# Get todo schema


class GetTodoSchema(BaseModel):
    id: Optional[Union[str, List[str]]] = Field(
        description="Id of the todo what we specially wants.")

## Creating Tools

In [65]:
from uuid import uuid4
from langchain_core.tools import tool


todos_data: List[TodoSchema] = []


@tool(args_schema=CreateTodoSchema)
def create_todo(name: str, description: Optional[str] = None, due_date: Optional[str] = None, priority: Optional[int] = None):
    """
    Create a new todo item.
   """
    todo = {
        "id": str(uuid4()),
        "name": name,
        "description": description,
        "due_date": due_date,
        "priority": priority
    }
    todos_data.append(TodoSchema(**todo))
    return {"success": "Todo is created successfully", "data": todo}


@tool(args_schema=UpdateTodoSchema)
def update_todo(id: str, description: Optional[str] = None, due_date: Optional[str] = None, priority: Optional[int] = None):
    """
    Update an existing todo item.
   """
    if len(todos_data) == 0:
        return {"error": "There is not todo available."}

    try:
        for index, todo in enumerate(todos_data):
            if todo.id == id:
                updated_data = todo.model_copy(update={
                    "description": description if description else todo.description,
                    "due_date": due_date if due_date else todo.due_date,
                    "priority": priority if priority else todo.priority
                })
                todos_data[index] = updated_data
                return {"success": "Todo is updated successfully", "data": updated_data.model_dump()}

        return {"error": "Todo not found."}
    except Exception as e:
        return {"error": str(e)}


@tool(args_schema=DeleteTodoSchema)
def delete_todo(id: Union[str, List[str]]):
    """
    Delete an existing todo item.
   """
    global todos_data

    if len(todos_data) == 0:
        return {"error": "There is not todo available."}

    ids = [id] if isinstance(id, str) else id
    todos_data = [todo for todo in todos_data if id in ids]
    if len(todos_data) == 0:
        return {"success": "Todo is deleted successfully."}
    else:
        return {"error": "Todo not found."}


@tool(args_schema=GetTodoSchema)
def get_todo(id: Optional[Union[str, List[str]]]):
    """
    Get an existing todo item.
   """
    if len(todos_data) == 0:
        return {"error": "There is no todo available."}

    if id is None:
        return [todo.model_dump for todo in todos_data]

    if isinstance(id, str):
        return next((todo for todo in todos_data if todo.id == id), None)
    else:
        return [todo.model_dump() for todo in todos_data if todo.id in id]


tools = [create_todo, update_todo, delete_todo, get_todo]
tools_with_name = {
    tool.name: tool for tool in tools}

## Initialize Model

In [66]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)
llm_with_tools = llm.bind_tools(tools)

## Create Chains 

In [71]:
from langchain_core.runnables import chain, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# Chat History
messages = []


def tool_calling(ai_message: AIMessage):
    tool_messages = []

    messages.append(ai_message)
    if len(ai_message.tool_calls) != 0:
        for tool in ai_message.tool_calls:
            selected_tool = tools_with_name[tool["name"]]
            tool_response = selected_tool.invoke(tool)

            tool_messages.append(tool_response)

    messages.extend(tool_messages)
    return messages


def get_model_response(list_messages):
    """
    finally send response back to the user.
    """
    if isinstance(list_messages[-1], ToolMessage):
        response = llm.invoke(list_messages)

        # Append Chat History.
        messages.append(response)
        return response
    else:
        return list_messages[-1]


@chain
def chain_executor(input):
    # Add the input to the messages.
    messages.append(HumanMessage(content=input))

    llm_chain = prompt | llm_with_tools | RunnableLambda(
        tool_calling) | RunnableLambda(get_model_response) | StrOutputParser()

    # Finally Send response back to the user.
    return llm_chain.invoke(HumanMessage(content=input))

## Main Execution Function

In [72]:
def main():
    while True:
        user_input = input("Specify your request.")
        if len(user_input) == 0:
            print("Error: Invalid request.")

        print(f"USER: {user_input}")

        llm_response = chain_executor.invoke(user_input)
        print(f"LLM: {llm_response}")

        choice = input("Do you want to continue. (y/n). ")
        if choice == "n" or choice == "no":
            break


main()

USER: Can you add a todo for going shopping and buy some cloths like jeans, t-shirts and cargos.
LLM: I have added a todo for going shopping to buy jeans, t-shirts, and cargos.
